In [2]:
from urllib.request import urlopen
import json
import pandas as pd
import geopandas as gpd
import plotly.express as px

In [ ]:
datapath = "../data/suitability_scores/"

# Load block group suitability scores
bg_suitability = pd.read_csv(datapath + "suitability_scores_bg.csv", dtype={"GEOID": str})
bg_suitability = bg_suitability.rename(columns={"GEOID": "fips"})

# # Load geojson file for block groups
# with urlopen("https://opendata.dc.gov/api/download/v1/items/c143846b7bf4438c954c5bb28e5d1a21/geojson?layers=2") as response:
#     bg_geojson = json.load(response)
    
bg_shapefile = gpd.read_file("C:/Users/limja/Downloads/2023 US Block Group Boundary File 500k/cb_2023_us_bg_500k.shp", dtype={"GEOID": str})
bg_json = bg_shapefile[['GEOID', 'geometry']].to_json()
bg_json = json.loads(bg_json)

# load a state level geojson file
with urlopen("https://raw.githubusercontent.com/PublicaMundi/MappingAPI/refs/heads/master/data/geojson/us-states.json") as response:
    states = json.load(response)

In [4]:
# get solar data 
solar = pd.read_csv("../data/projects/solar/solar_raw.csv")

# Get weighted average of all factors

**Formulas:**
1. Avg of all factors = (sum of all factors) / (number of factors)
2. Gem Model weighted avg
    - 2 slope
    - 1 land cover
    - 2 population density
    - 2 habitat
    - 4 protected land
    - 1 distance to substation
    - 4 GHI (in the default GEM they used `Solar PV: 1-Axis Tracking Flat-Plate Collector`)

In [5]:
# Formula 1
factors = ['GHI', 'Protected_Land', 'Habitat', 'Slope', 'Population_Density', 'Distance_to_Substation', 'Land_Cover']

# sum the factors
formula_1 = bg_suitability[factors].sum(axis=1).div(len(factors))
df_formula_1 = pd.DataFrame({"fips": bg_suitability['fips'], "suitability": formula_1})

# Formula 2
factors_plus_weights = {
    'GHI': 4,
    'Protected_Land': 4,
    'Habitat': 2,
    'Slope': 2,
    'Population_Density': 2,
    'Distance_to_Substation': 1,
    'Land_Cover': 1
}

formula_2 = bg_suitability.copy()
# Multiply each factor by its weight
for factor, weight in factors_plus_weights.items():
    formula_2[factor] = formula_2[factor] * weight
    
# sum the factors
formula_2 = formula_2[factors].sum(axis=1).div(sum(factors_plus_weights.values()))
df_formula_2 = pd.DataFrame({"fips": bg_suitability['fips'], "suitability": formula_2})

In [9]:
import json
from shapely.geometry import shape, mapping

def simplify_geojson(geojson_data, tolerance=0.01):
    """
    Simplifies the geometries in a GeoJSON file.

    :param input_path: Path to the input GeoJSON file.
    :param output_path: Path to save the simplified GeoJSON file.
    :param tolerance: The tolerance for simplification. Increase for more simplification.
    """

    for feature in geojson_data.get('features', []):
        geom = shape(feature['geometry'])
        # simplify the geometry; set preserve_topology=True to keep valid geometries
        simplified_geom = geom.simplify(tolerance, preserve_topology=True)
        feature['geometry'] = mapping(simplified_geom)

    return geojson_data

# Simplify the geojson
bg_json_simplified = simplify_geojson(bg_json, tolerance=0.05)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# custom color scale
pubu_color_scale = [
    (0.00, "#f7fcfd"),
    (0.40, "#e0ecf4"),
    (0.50, "#bfd3e6"),
    (0.60, "#9ebcda"),
    (0.70, "#8c96c6"),
    (0.80, "#8c6bb1"),
    (0.90, "#88419d"),
    (1.00, "#4a1486")
]


# Create the choropleth map
fig = px.choropleth(
    df_formula_2,
    geojson=bg_json,
    locations='fips',  # Column in df that matches GeoJSON IDs
    featureidkey="properties.GEOID",  # Path to field in GeoJSON feature object with which to match the values passed in to locations
    color='suitability',  # Column to plot
    color_continuous_scale=pubu_color_scale,  # Custom color scale
    range_color=(0, 100),
    scope="usa",  # Focus on the United States    
)

fig.update_traces(marker_line_width=0, hoverinfo="skip")  # Remove bg borders

# Overlay state boundaries
fig.add_trace(go.Choropleth(
    geojson=states,  # Use the state-level GeoJSON
    locations=[feature["id"] for feature in states["features"]],  # State IDs
    z=[0] * len(states["features"]),  # Dummy data to avoid coloring
    colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0)"]],  # Transparent fill
    showscale=False,
    marker=dict(line=dict(color='black', width=1.5)),  # Black state boundaries
))

# Update layout and map appearance
fig.update_geos(
    fitbounds="locations",  # Auto-zoom based on data
    visible=False  # Hide base map
)

fig.update_layout(
    margin={"r": 0, "t": 0, "l": 0, "b": 0}, clickmode='none',  
)

# # add state points px.scatter_geo(solar, lat="latitude", lon="longitude", scope="usa", color="statename")
len_scatter = len(px.scatter_geo(solar, lat="latitude", lon="longitude", scope="usa", size="total_mw").data)
for i in range(len_scatter):
    fig.add_trace(
        px.scatter_geo(solar, lat="latitude", lon="longitude", scope="usa", size="total_mw", color_discrete_sequence=px.colors.qualitative.Light24).data[i]
    )

# Don't show the legend for the scatter points
fig.update_layout(showlegend=False)


fig.show()